# Implantar um modelo de propensão de rotatividade de usuários do BigQuery ML no Vertex AI para previsões on-line

## Objetivos de aprendizado

* Explore e pré-processe uma amostra de dados do [Google Analytics 4](https://support.google.com/analytics/answer/7029846) no [BigQuery](https://cloud.google.com/bigquery) para aprendizado de máquina.
* Treine um classificador do [BigQuery ML (BQML)](https://cloud.google.com/bigquery-ml) [XGBoost](https://xgboost.readthedocs.io/en/latest/) para prever a perda de usuários um aplicativo de jogos para celular.
* Ajuste um classificador BQML XGBoost usando [recursos de ajuste de hiperparâmetro BQML](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree).
* Avaliar o desempenho de um classificador BQML XGBoost.
* Explique seu modelo XGBoost com atribuições de recursos globais de [BQML Explainable AI](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview).
* Gere previsões em lote com seu modelo BQML XGBoost.
* Exporte um modelo BQML XGBoost para um [Google Cloud Storage](https://cloud.google.com/storage).
* Carregue e implante um modelo BQML XGBoost em um endpoint [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) para previsões on-line.

## Introdução

Neste laboratório, você treinará, avaliará, explicará e gerará previsões on-line e em lote com um modelo XGBoost do BigQuery ML (BQML). Você usará um conjunto de dados do Google Analytics 4 de um aplicativo móvel real, Flood it! ([aplicativo Android](https://play.google.com/store/apps/details?id=com.labpixies.flood), [aplicativo iOS](https://itunes.apple.com/us/app/ flood-it!/id476943146?mt=8)), para determinar a probabilidade de os usuários retornarem ao aplicativo. Você gerará previsões em lote com seu modelo do BigQuery ML, além de exportá-lo e implantá-lo no **Vertex AI** para previsões on-line.

O [BigQuery ML](https://cloud.google.com/bigquery-ml/docs/introduction) permite treinar e fazer inferência em lote com modelos de aprendizado de máquina no BigQuery usando consultas SQL padrão mais rapidamente, eliminando a necessidade de mover dados com menos linhas de código. [Vertex AI](https://cloud.google.com/vertex-ai) é a plataforma unificada complementar de próxima geração do Google Cloud para desenvolvimento de aprendizado de máquina. Ao desenvolver e implantar soluções de aprendizado de máquina BQML na Vertex AI, você pode aproveitar um serviço de previsão online escalável e ferramentas de MLOps para treinamento e monitoramento de modelos para melhorar significativamente sua produtividade de desenvolvimento, a capacidade de dimensionar seu fluxo de trabalho e a tomada de decisões com seus dados e acelerar tempo para valorizar.

![BQML Vertex AI](./images/vertex-bqml-lab-architecture-diagram.png "Vertex BQML Lab Architecture Diagram")

Observação: este laboratório é inspirado e estende a [previsão de rotatividade para desenvolvedores de jogos usando o Google Analytics 4 (GA4) e o BigQuery ML](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game -desenvolvedores-usando-google-analytics-4-ga4-and-bigquery-ml). Consulte a postagem do blog e o tutorial que o acompanha para obter mais detalhes sobre este caso de uso e o BigQuery ML. Neste laboratório, você dará um passo adiante e se concentrará em como a Vertex AI estende os recursos do BQML por meio da previsão on-line para que você possa incorporar as previsões de rotatividade de clientes nas UIs de tomada de decisão, como [painéis do Looker](https://looker.com/google -cloud), mas também previsões on-line diretamente nos aplicativos do cliente para impulsionar intervenções direcionadas, como incentivos direcionados.

### Caso de uso: modelagem de propensão à rotatividade de usuários no setor de jogos para dispositivos móveis

De acordo com um [estudo de 2019](https://gameanalytics.com/reports/mobile-gaming-industry-analysis-h1-2019) sobre 100 mil jogos para celular da Mobile Gaming Industry Analysis, a maioria dos jogos para celular tem apenas 25% de retenção taxa para usuários após as primeiras 24 horas, sabe-se que qualquer jogo "abaixo de 30% de retenção geralmente precisa de melhorias". Para desenvolvedores de jogos para dispositivos móveis, melhorar a retenção de usuários é fundamental para a estabilidade da receita e o aumento da lucratividade. Na verdade, [pesquisa da Bain & Company](https://hbr.org/2014/10/the-value-of-keeping-the-right-customers) descobriu que um crescimento de 5% na taxa de retenção pode resultar em 25% a 95% nos lucros. Com custos mais baixos para reter os clientes existentes, o objetivo comercial dos desenvolvedores de jogos é claro: reduzir a rotatividade e melhorar a fidelidade do cliente para gerar lucratividade a longo prazo.

Sua tarefa neste laboratório: usar o aprendizado de máquina para prever a propensão de desligamento do usuário após o primeiro dia, uma janela crucial de integração do usuário, e fornecer essas previsões on-line para informar intervenções como recompensas e notificações direcionadas no jogo.

## Configuração

### Definição de Constantes

In [ ]:
# Recupere e defina as variáveis de ambiente PROJECT_ID e REGION.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [ ]:
BQ_LOCATION = 'US'
REGION = 'us-central1'

### Importe as bibliotecas

In [ ]:
from google.cloud import bigquery
from google.cloud import aiplatform as vertexai
import numpy as np
import pandas as pd

### Criar um bucket para o artifact storage

Crie um bucket globalmente exclusivo do Google Cloud Storage para armazenamento de artefatos. Você usará esse bucket para exportar seu modelo BQML posteriormente no laboratório e carregá-lo no Vertex AI.

In [ ]:
GCS_BUCKET = f"{PROJECT_ID}-bqmlga4"

In [ ]:
!gsutil mb -l $REGION gs://$GCS_BUCKET

### Criar um conjunto de dados do BigQuery

Em seguida, crie um conjunto de dados do BigQuery a partir deste notebook usando o [utilitário de linha de comando `bq` baseado em Python](https://cloud.google.com/bigquery/docs/bq-command-line-tool).

Esse conjunto de dados agrupará suas visualizações de recursos, modelo e tabela de previsões. Você pode visualizá-lo no console do [BigQuery](https://pantheon.corp.google.com/bigquery).

In [ ]:
BQ_DATASET = f"{PROJECT_ID}:bqmlga4"

In [ ]:
!bq mk --location={BQ_LOCATION} --dataset {BQ_DATASET}

### Inicialize o cliente Vertex Python SDK

Importe o Vertex SDK for Python para seu ambiente Python e inicialize-o.

In [ ]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{GCS_BUCKET}")

## Análise de dados exploratórios (EDA) no BigQuery

Este laboratório usa um [conjunto de dados público do BigQuery]() que contém dados brutos de eventos de um aplicativo de jogos para dispositivos móveis real chamado **Flood it!** ([Aplicativo Android](https://play.google.com/store/apps/details?id=com.labpixies.flood), [iOS app](https://itunes.apple.com/us/app/flood-it!/id476943146?mt=8)).

O esquema de dados é originário do Google Analytics para Firebase, mas é o mesmo esquema do [Google Analytics 4](https://support.google.com/analytics/answer/9358801).

Dê uma olhada em uma amostra do conjunto de dados de eventos brutos usando a consulta abaixo:

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT 
    *
FROM
  `firebase-public-project.analytics_153293282.events_*`
    
TABLESAMPLE SYSTEM (1 PERCENT)

Nota: na célula acima, o Jupyterlab executa células começando com `%%bigquery` como consultas SQL.

O Google Analytics 4 usa um modelo de medição baseado em eventos e cada linha nesse conjunto de dados é um evento. Veja o [esquema completo](https://support.google.com/analytics/answer/7029846) e detalhes sobre cada coluna. Como você pode ver acima, certas colunas são registros aninhados e contêm informações detalhadas, como:

* app_info
* device
* ecommerce
* event_params
* geo
* traffic_source
* user_properties
* items*
* web_info*

Este conjunto de dados contém 5,7 milhões de eventos de mais de 15 mil usuários.

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT 
    COUNT(DISTINCT user_pseudo_id) as count_distinct_users,
    COUNT(event_timestamp) as count_events
FROM
  `firebase-public-project.analytics_153293282.events_*`

## Preparação do conjunto de dados no BigQuery

Agora que você tem uma noção melhor do conjunto de dados com o qual trabalhará, passará pela transformação de dados brutos de eventos em um conjunto de dados adequado para aprendizado de máquina usando comandos SQL no BigQuery. Especificamente, você irá:

* Eventos agregados para que cada linha represente um ID de usuário exclusivo separado.
* Defina o recurso **user churn label** para treinar seu modelo para previsão (por exemplo, 1 = cancelado, 0 = retornado).
* Crie recursos **demográficos do usuário**.
* Crie recursos **comportamentais do usuário** a partir de eventos de aplicativos agregados.

### Definindo rotatividade para cada usuário

Há muitas maneiras de definir a perda de usuários, mas para os propósitos deste laboratório, você preverá a perda de um dia de usuários que não voltam e usam o aplicativo novamente após 24 horas do primeiro engajamento do usuário. Isso serve para capturar a rotatividade após a "primeira impressão" de um usuário do aplicativo ou da experiência de integração.

Em outras palavras, após 24 horas do primeiro envolvimento de um usuário com o aplicativo:

* se o usuário não mostrar nenhum dado de evento depois disso, o usuário será considerado **desconectado**.
* se o usuário tiver pelo menos um ponto de dados de evento depois disso, o usuário será considerado **retornado**.

Você também pode remover usuários que provavelmente nunca retornaram depois de passar apenas alguns minutos com o aplicativo, que às vezes é chamado de "rejeição". Por exemplo, você criará seu modelo apenas em usuários que passaram pelo menos 10 minutos com o aplicativo (usuários que não desistiram).

A consulta abaixo define um usuário churned com a seguinte definição:

**Rotatividade = "qualquer usuário que passou pelo menos 10 minutos no aplicativo, mas depois de 24 horas desde o primeiro contato com o aplicativo, nunca mais usou o aplicativo"**

Você usará os dados brutos do evento, desde o primeiro toque (instalação do aplicativo) até o último toque, para identificar usuários cancelados e rejeitados na consulta de visualização `user_churn` abaixo:

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_churn AS (
  WITH firstlasttouch AS (
    SELECT
      user_pseudo_id,
      MIN(event_timestamp) AS user_first_engagement,
      MAX(event_timestamp) AS user_last_engagement
    FROM
      `firebase-public-project.analytics_153293282.events_*`
    WHERE event_name="user_engagement"
    GROUP BY
      user_pseudo_id

  )
  
SELECT
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,

    #adicione 24 horas ao primeiro toque do usuário
    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,
    
    #rotatividade = 1 se last_touch dentro de 24 horas da instalação do aplicativo, senão 0
    IF (user_last_engagement < (user_first_engagement + 86400000000),
    1,
    0 ) AS churned,
    
    #retornado = 1 se last_touch dentro de 10 min, senão 0
    IF (user_last_engagement <= (user_first_engagement + 600000000),
    1,
    0 ) AS bounced,
  FROM
    firstlasttouch
  GROUP BY
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement
    );

SELECT 
  * 
FROM 
  bqmlga4.user_churn 
LIMIT 100;

Analise quantos dos 15 mil usuários rejeitaram e retornaram abaixo:

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
    bounced,
    churned, 
    COUNT(churned) as count_users
FROM
    bqmlga4.user_churn
GROUP BY 
  bounced,
  churned
ORDER BY 
  bounced

Para os dados de treinamento, você acabará usando apenas dados onde rejeitados = 0. Com base nos 15 mil usuários, você pode ver que 5.557 (cerca de 41%) usuários rejeitaram nos primeiros dez minutos de seu primeiro envolvimento com o aplicativo. Dos 8.031 usuários restantes, 1.883 usuários (cerca de 23%) desistiram após 24 horas, o que você pode validar com a consulta abaixo:

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
    COUNTIF(churned=1)/COUNT(churned) as churn_rate
FROM
    bqmlga4.user_churn
WHERE bounced = 0

### Extraia recursos demográficos do usuário

Há várias informações demográficas de usuários incluídas neste conjunto de dados, incluindo `app_info`, `device`, `ecommerce`, `event_params` e `geo`. Os recursos demográficos podem ajudar o modelo a prever se os usuários em determinados dispositivos ou países têm maior probabilidade de sair.

Observe que os dados demográficos de um usuário podem mudar ocasionalmente (por exemplo, mudar de país). Para simplificar, você usará as informações demográficas que o Google Analytics 4 fornece quando o usuário interagiu pela primeira vez com o aplicativo, conforme indicado por MIN(event_timestamp) na consulta abaixo. Isso permite que cada usuário único seja representado por uma única linha.

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_demographics AS (

  WITH first_values AS (
      SELECT
          user_pseudo_id,
          geo.country as country,
          device.operating_system as operating_system,
          device.language as language,
          ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp DESC) AS row_num
      FROM `firebase-public-project.analytics_153293282.events_*`
      WHERE event_name="user_engagement"
      )
  SELECT * EXCEPT (row_num)
  FROM first_values
  WHERE row_num = 1
  );

SELECT
  *
FROM
  bqmlga4.user_demographics
LIMIT 10

### Agregar recursos comportamentais do usuário

Os dados comportamentais nos dados brutos do evento abrangem vários eventos - e, portanto, linhas - por usuário. O objetivo desta seção é agregar e extrair dados comportamentais para cada usuário, resultando em uma linha de dados comportamentais por usuário único.


Como primeiro passo, você pode explorar todos os eventos exclusivos que existem neste conjunto de dados, com base em event_name:

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  event_name,
  COUNT(event_name) as event_count
FROM
    `firebase-public-project.analytics_153293282.events_*`
GROUP BY 
  event_name
ORDER BY
   event_count DESC

Para este laboratório, para prever se um usuário sairá ou retornará, você pode começar contando o número de vezes que um usuário se envolve nos seguintes tipos de evento:

* user_engagement
* level_start_quickplay
* level_end_quickplay
* level_complete_quickplay
* level_reset_quickplay
* post_score
* spend_virtual_currency
* ad_reward
* challenge_a_friend
* completed_5_levels
* use_extra_steps

Na consulta SQL abaixo, você agregará os dados comportamentais calculando o número total de vezes em que cada um dos event_names acima ocorreu no conjunto de dados por usuário.

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.user_behavior AS (
WITH
  events_first24hr AS (
    # Select user data only from first 24 hr of using the app.
    SELECT
      e.*
    FROM
      `firebase-public-project.analytics_153293282.events_*` e
    JOIN
      bqmlga4.user_churn c
    ON
      e.user_pseudo_id = c.user_pseudo_id
    WHERE
      e.event_timestamp <= c.ts_24hr_after_first_engagement
    )
SELECT
  user_pseudo_id,
  SUM(IF(event_name = 'user_engagement', 1, 0)) AS cnt_user_engagement,
  SUM(IF(event_name = 'level_start_quickplay', 1, 0)) AS cnt_level_start_quickplay,
  SUM(IF(event_name = 'level_end_quickplay', 1, 0)) AS cnt_level_end_quickplay,
  SUM(IF(event_name = 'level_complete_quickplay', 1, 0)) AS cnt_level_complete_quickplay,
  SUM(IF(event_name = 'level_reset_quickplay', 1, 0)) AS cnt_level_reset_quickplay,
  SUM(IF(event_name = 'post_score', 1, 0)) AS cnt_post_score,
  SUM(IF(event_name = 'spend_virtual_currency', 1, 0)) AS cnt_spend_virtual_currency,
  SUM(IF(event_name = 'ad_reward', 1, 0)) AS cnt_ad_reward,
  SUM(IF(event_name = 'challenge_a_friend', 1, 0)) AS cnt_challenge_a_friend,
  SUM(IF(event_name = 'completed_5_levels', 1, 0)) AS cnt_completed_5_levels,
  SUM(IF(event_name = 'use_extra_steps', 1, 0)) AS cnt_use_extra_steps,
FROM
  events_first24hr
GROUP BY
  user_pseudo_id
  );

SELECT
  *
FROM
  bqmlga4.user_behavior
LIMIT 10

### Prepare seus conjuntos de dados de treinamento/avaliação/teste para aprendizado de máquina

Nesta seção, agora você pode combinar essas três visualizações intermediárias (`user_churn`, `user_demographics` e `user_behavior`) na visualização final de dados de treinamento chamada `ml_features`. Aqui você também pode especificar rejeição = 0, para limitar os dados de treinamento apenas aos usuários que não "rejeitam" nos primeiros 10 minutos de uso do aplicativo.

Observe na consulta abaixo que uma coluna `data_split` manual é criada na tabela do BigQuery ML usando [funções de hashing do BigQuery](https://towardsdatascience.com/ml-design-pattern-5-repeatable-sampling-c0ccb2889f39) para amostragem repetitiva. Especifica 80% de treinamento | 10% de avaliação | 20% de teste para avaliar o desempenho e a generalização do seu modelo.

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE VIEW bqmlga4.ml_features AS (
    
  SELECT
    dem.user_pseudo_id,
    IFNULL(dem.country, "Unknown") AS country,
    IFNULL(dem.operating_system, "Unknown") AS operating_system,
    IFNULL(REPLACE(dem.language, "-", "X"), "Unknown") AS language,
    IFNULL(beh.cnt_user_engagement, 0) AS cnt_user_engagement,
    IFNULL(beh.cnt_level_start_quickplay, 0) AS cnt_level_start_quickplay,
    IFNULL(beh.cnt_level_end_quickplay, 0) AS cnt_level_end_quickplay,
    IFNULL(beh.cnt_level_complete_quickplay, 0) AS cnt_level_complete_quickplay,
    IFNULL(beh.cnt_level_reset_quickplay, 0) AS cnt_level_reset_quickplay,
    IFNULL(beh.cnt_post_score, 0) AS cnt_post_score,
    IFNULL(beh.cnt_spend_virtual_currency, 0) AS cnt_spend_virtual_currency,
    IFNULL(beh.cnt_ad_reward, 0) AS cnt_ad_reward,
    IFNULL(beh.cnt_challenge_a_friend, 0) AS cnt_challenge_a_friend,
    IFNULL(beh.cnt_completed_5_levels, 0) AS cnt_completed_5_levels,
    IFNULL(beh.cnt_use_extra_steps, 0) AS cnt_use_extra_steps,
    chu.user_first_engagement,
    chu.month,
    chu.julianday,
    chu.dayofweek,
    chu.churned,
    # https://towardsdatascience.com/ml-design-pattern-5-repeatable-sampling-c0ccb2889f39
    # BQML Hyperparameter tuning requires STRING 3 partition data_split column.
    # 80% 'TRAIN' | 10%'EVAL' | 10% 'TEST'    
    CASE
      WHEN ABS(MOD(FARM_FINGERPRINT(dem.user_pseudo_id), 10)) <= 7
        THEN 'TRAIN'
      WHEN ABS(MOD(FARM_FINGERPRINT(dem.user_pseudo_id), 10)) = 8
        THEN 'EVAL'
      WHEN ABS(MOD(FARM_FINGERPRINT(dem.user_pseudo_id), 10)) = 9
        THEN 'TEST'    
          ELSE '' END AS data_split
  FROM
    bqmlga4.user_churn chu
  LEFT OUTER JOIN
    bqmlga4.user_demographics dem
  ON 
    chu.user_pseudo_id = dem.user_pseudo_id
  LEFT OUTER JOIN 
    bqmlga4.user_behavior beh
  ON
    chu.user_pseudo_id = beh.user_pseudo_id
  WHERE chu.bounced = 0
  );

SELECT
  *
FROM
  bqmlga4.ml_features
LIMIT 10

### Validar divisões de recursos

Run the query below to validate the number of examples in each data partition for the 80% train |10% eval |10% test split.

Execute a consulta abaixo para validar o número de exemplos em cada partição de dados para  80% treinamento |10% avaliação |10% teste.

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  data_split,
  COUNT(*) AS n_examples
FROM bqmlga4.ml_features
GROUP BY data_split

## Treine e ajuste um modelo de propensão BQML XGBoost para prever a perda de clientes

O código a seguir treina e ajusta os hiperparâmetros para um modelo XGBoost. PARA fornecer uma demonstração mínima de ajuste de hiperparâmetros BQML neste laboratório, este modelo levará cerca de 18 minutos para treinar e ajustar com seu espaço de pesquisa restrito e baixo número de tentativas. Na prática, você geralmente precisa de [pelo menos 10 testes por hiperparâmetro](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-hyperparameter-tuning#how_many_trials_do_i_need_to_tune_a_model) para obter melhores resultados .

Para obter mais informações sobre os hiperparâmetros padrão usados, você pode ler a documentação:
[Instrução CREATE MODEL para modelos de árvore otimizada usando XGBoost](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)

|Model   | BQML model_type | Vantagens | Desvantagens| 
|:-------|:----------:|:----------:|-------------:|
|XGBoost |     BOOSTED_TREE_CLASSIFIER [(documentation)](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)       |   Alto desempenho do modelo com importâncias de recursos e explicabilidade | Mais lento para treinar do que BQML LOGISTIC_REG |

Observação: quando você executa a instrução CREATE MODEL, o BigQuery ML pode dividir automaticamente seus dados em treinamento e teste para que você possa avaliar imediatamente o desempenho do seu modelo após o treinamento. Esta é uma ótima opção para prototipagem rápida de modelos. Neste laboratório, no entanto, você dividirá seus dados manualmente acima usando hash para divisões de dados reproduzíveis que podem ser usadas comparando avaliações de modelo em diferentes execuções.

In [ ]:
MODEL_NAME="churn_xgb"

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL bqmlga4.churn_xgb

OPTIONS(
  MODEL_TYPE="BOOSTED_TREE_CLASSIFIER",
  # Declare label column.
  INPUT_LABEL_COLS=["churned"],
  # Specify custom data splitting using the `data_split` column.
  DATA_SPLIT_METHOD="CUSTOM",
  DATA_SPLIT_COL="data_split",
  # Enable Vertex Explainable AI aggregated feature attributions.
  ENABLE_GLOBAL_EXPLAIN=True,
  # Hyperparameter tuning arguments.
  num_trials=8,
  max_parallel_trials=4,
  HPARAM_TUNING_OBJECTIVES=["roc_auc"],
  EARLY_STOP=True,
  # Hyperpameter search space.
  LEARN_RATE=HPARAM_RANGE(0.01, 0.1),
  MAX_TREE_DEPTH=HPARAM_CANDIDATES([5,6])
) AS

SELECT
  * EXCEPT(user_pseudo_id)
FROM
  bqmlga4.ml_features

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT *
FROM
  ML.TRIAL_INFO(MODEL `bqmlga4.churn_xgb`);

## Avalie o desempenho do modelo BQML XGBoost

Quando o treinamento terminar, execute [ML.EVALUATE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) para retornar as métricas de avaliação do modelo. Por padrão, todas as avaliações do modelo serão retornadas, portanto, a consulta abaixo apenas retorna o desempenho do modelo para uma primeira avaliação ideal.

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlga4.churn_xgb)
WHERE trial_id=1;

ML.EVALUATE gera o [precision, recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall), [accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy), [log_loss](https://en.wikipedia.org/wiki/Loss_functions_for_classification#Logistic_loss), [f1_score](https://en.wikipedia.org/wiki/F-score) e [roc_auc](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc) usando o limite de classificação padrão de 0,5, que pode ser modificado por usando o parâmetro opcional `THRESHOLD`.

Em seguida, use a função [ML.CONFUSION_MATRIX](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-confusion) para retornar uma matriz de confusão para o modelo de classificação de entrada e dados de entrada.

Para obter mais informações sobre matrizes de confusão, leia uma explicação detalhada [aqui](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative).

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  expected_label,
  _0 AS predicted_0,
  _1 AS predicted_1
FROM
  ML.CONFUSION_MATRIX(MODEL bqmlga4.churn_xgb)
WHERE trial_id=1;

Você também pode plotar a curva AUC-ROC usando [ML.ROC_CURVE](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-roc) para retornar as métricas para diferentes valores limite para o modelo.

In [ ]:
%%bigquery df_roc --project $PROJECT_ID

SELECT * FROM ML.ROC_CURVE(MODEL bqmlga4.churn_xgb) WHERE trial_id=1;

In [ ]:
df_roc.plot(x="false_positive_rate", y="recall", title="AUC-ROC curve")

## Inspecione as atribuições de recursos globais

Para fornecer mais contexto ao desempenho do seu modelo, você pode usar o [ML.GLOBAL_EXPLAIN](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-global-explain#get_global_feature_importance_for_each_class_of_a_boosted_tree_classifier_model ) que aproveita o Vertex Explainable AI como back-end. [Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai) ajuda você a entender as saídas do seu modelo para tarefas de classificação e regressão. Especificamente, o Vertex AI informa o quanto cada recurso nos dados contribuiu para o resultado previsto do seu modelo. Você pode usar essas informações para verificar se o modelo está se comportando conforme o esperado, identificar e mitigar vieses em seus modelos e obter ideias de maneiras de melhorar seu modelo e seus dados de treinamento.

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.GLOBAL_EXPLAIN(MODEL bqmlga4.churn_xgb)
ORDER BY
  attribution DESC;

## Gerar previsões em lote

Você pode gerar previsões em lote para seu modelo BQML XGBoost usando [ML.PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict).

In [ ]:
%%bigquery --project $PROJECT_ID

SELECT
  *
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.ml_features WHERE data_split = "TEST"))

A consulta a seguir retorna a probabilidade de o usuário retornar após 24 horas. Quanto maior a probabilidade e mais próximo de 1, maior a probabilidade de o usuário se desligar, e quanto mais próximo de 0, maior a probabilidade de o usuário retornar.

In [ ]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE TABLE bqmlga4.churn_predictions AS (
SELECT
  user_pseudo_id,
  churned,
  predicted_churned,
  predicted_churned_probs[OFFSET(0)].prob as probability_churned
FROM
  ML.PREDICT(MODEL bqmlga4.churn_xgb,
  (SELECT * FROM bqmlga4.ml_features))
);

## Exporte um modelo BQML para o Vertex AI para previsões online

Consulte o Guia oficial do BigQuery ML: [Exportar um modelo do BigQuery ML para previsão on-line](https://cloud.google.com/bigquery-ml/docs/export-model-tutorial) para mais detalhes.

### Exportar modelo BQML para GCS

Você usará o comando `bq extract` na ferramenta de linha de comando `bq` para exportar seus recursos do modelo BQML XGBoost para o Google Cloud Storage para persistência. Consulte a [documentação](https://cloud.google.com/bigquery-ml/docs/exporting-models) para opções adicionais de exportação de modelos.

In [ ]:
BQ_MODEL = f"{BQ_DATASET}.{MODEL_NAME}"
BQ_MODEL_EXPORT_DIR = f"gs://{GCS_BUCKET}/{MODEL_NAME}"

In [ ]:
!bq --location=$BQ_LOCATION extract \
--destination_format ML_XGBOOST_BOOSTER \
--model $BQ_MODEL \
$BQ_MODEL_EXPORT_DIR

Navegue até [Google Cloud Storage](https://pantheon.corp.google.com/storage) no Google Cloud Console para `"gs://{GCS_BUCKET}/{MODEL_NAME}"`. Valide se você vê seus ativos de modelo exportados no formato abaixo:

```
|--/{GCS_BUCKET}/{MODEL_NAME}/
   |--/assets/                       # Contains preprocessing code.  
      |--0_categorical_label.txt     # Contains country vocabulary.
      |--1_categorical_label.txt     # Contains operating_system vocabulary.
      |--2_categorical_label.txt     # Contains language vocabulary.
      |--model_metadata.json         # contains model feature and label mappings.
   |--main.py                        # Can be called for local training runs.
   |--model.bst                      # XGBoost saved model format.
   |--xgboost_predictor-0.1.tar.gz   # Compress XGBoost model with prediction function. 
```

### Carregue o modelo BQML para o Vertex AI do GCS

O Vertex AI contém contêineres otimizados de treinamento e previsão pré-criados para estruturas populares de ML, como TensorFlow, Pytorch e XGBoost. Você fará o upload do seu XGBoost do GCS para o Vertex AI e fornecerá o [mais recente contêiner de previsão Vertex XGBoost pré-criado](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers) para executar seu código de modelo para gerar previsões nas células abaixo.

In [ ]:
IMAGE_URI='us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-4:latest'

In [ ]:
model = vertexai.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri=BQ_MODEL_EXPORT_DIR,
    serving_container_image_uri=IMAGE_URI,
)

### Implante um Vertex `Endpoint` para previsões online

Antes de usar seu modelo para fazer previsões, você precisa implantá-lo em um objeto `Endpoint`. Ao implantar um modelo em um `endpoint`, você associa recursos físicos (de máquina) a esse modelo para permitir que ele atenda a previsões online. As previsões online têm requisitos de baixa latência; fornecer recursos para o modelo com antecedência reduz a latência. Você pode fazer isso chamando a função deploy no recurso `Model`. Isso fará duas coisas:

1. Crie um recurso `Endpoint` para implantar o recurso `Model`.
2. Implante o recurso `Model` no recurso `Endpoint`.

A função `deploy()` recebe os seguintes parâmetros:

* `deployed_model_display_name`: Um nome legível para o modelo implementado.
* `traffic_split`: Porcentagem de tráfego no endpoint que vai para este modelo, que é especificado como um dicionário de um ou mais pares de chave/valor. Se for apenas um modelo, especifique como { "0": 100 }, em que "0" se refere a esse modelo sendo carregado e 100 significa 100% do tráfego.
* `machine_type`: O tipo de máquina a ser usada para treinamento.
* `accelerator_type`: O tipo de acelerador de hardware.
* `accelerator_count`: O número de aceleradores a serem anexados a uma réplica do trabalhador.
* `starting_replica_count`: o número de instâncias de computação a serem provisionadas inicialmente.
* `max_replica_count`: o número máximo de instâncias de computação para escalar. Neste laboratório, apenas uma instância é provisionada.
* `explanation_parameters`: Metadados para configurar o método de aprendizado Explainable AI.
* `explanation_metadata`: Metadados que descrevem seu modelo TensorFlow para IA explicativa, como recursos, tensores de entrada e saída.

Observação: isso pode levar cerca de 3 a 5 minutos para provisionar recursos de previsão para seu modelo.

In [ ]:
endpoint = model.deploy(
    traffic_split={"0": 100},
    machine_type="n1-standard-2",
)

### Modelo de consulta para previsões on-line

O XGBoost aceita apenas entradas de recursos numéricos. Quando você treinou seu modelo BQML acima com a instrução CREATE MODEL, ele tratou automaticamente a codificação de recursos categóricos como `país` do usuário, `sistema operacional` e `idioma` em representações numéricas. Para que nosso modelo exportado gere previsões on-line, você usará os arquivos de vocabulário de recursos categóricos exportados na pasta `assets/` do diretório de seu modelo e o código de pré-processamento Scikit-Learn abaixo para mapear suas instâncias de teste para valores numéricos.

In [ ]:
CATEGORICAL_FEATURES = ['country',
                        'operating_system',
                        'language']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
def _build_cat_feature_encoders(cat_feature_list, gcs_bucket, model_name, na_value='Unknown'):
    """Build categorical feature encoders for mapping text to integers for XGBoost inference. 
    Args:
      cat_feature_list (list): List of string feature names.
      gcs_bucket (str): A string path to your Google Cloud Storage bucket.
      model_name (str): A string model directory in GCS where your BQML model was exported to.
      na_value (str): default is 'Unknown'. String value to replace any vocab NaN values prior to encoding.
    Returns:
      feature_encoders (dict): A dictionary containing OrdinalEncoder objects for integerizing 
        categorical features that has the format [feature] = feature encoder.
    """
    """Construa codificadores de recursos categóricos para mapear texto para inteiros para inferência do XGBoost.
     Args:
       cat_feature_list (list): Lista de nomes de recursos de string.
       gcs_bucket (str): um caminho de string para seu bucket do Google Cloud Storage.
       model_name (str): Um diretório de modelo de string no GCS para o qual seu modelo BQML foi exportado.
       na_value (str): o padrão é 'Desconhecido'. Valor de string para substituir quaisquer valores de vocab NaN antes da codificação.
     Devoluções:
       feature_encoders (dict): Um dicionário contendo objetos OrdinalEncoder para inteiros
         feições categóricas que tem o formato [feature] = codificador de feições.
     """
    
    feature_encoders = {}
    
    for idx, feature in enumerate(cat_feature_list):
        feature_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
        feature_vocab_file = f"gs://{gcs_bucket}/{model_name}/assets/{idx}_categorical_label.txt"
        feature_vocab_df = pd.read_csv(feature_vocab_file, delimiter = "\t", header=None).fillna(na_value)
        feature_encoder.fit(feature_vocab_df.values)
        feature_encoders[feature] = feature_encoder
    
    return feature_encoders

In [ ]:
def preprocess_xgboost(instances, cat_feature_list, feature_encoders):
    """Transform instances to numerical values for inference.
    Args:
      instances (list[dict]): A list of feature dictionaries with the format feature: value. 
      cat_feature_list (list): A list of string feature names.
      feature_encoders (dict): A dictionary with the format feature: feature_encoder.
    Returns:
      transformed_instances (list[list]): A list of lists containing numerical feature values needed
        for Vertex XGBoost inference.
    """

    """Transforme instâncias em valores numéricos para inferência.
     Args:
       instâncias (list[dict]): Uma lista de dicionários de recursos com o formato feature: value.
       cat_feature_list (list): Uma lista de nomes de recursos de string.
       feature_encoders (dict): Um dicionário com o formato feature: feature_encoder.
     Devoluções:
       transformado_instances (list[list]): Uma lista de listas contendo valores numéricos de recursos necessários
         para inferência Vertex XGBoost.
     """
     
    transformed_instances = []
    
    for instance in instances:
        for feature in cat_feature_list:
            feature_int = feature_encoders[feature].transform([[instance[feature]]]).item()
            instance[feature] = feature_int
            instance_list = list(instance.values())
        transformed_instances.append(instance_list)

    return transformed_instances

In [ ]:
# Crie um dicionário de codificadores de recursos categóricos ordinais.
feature_encoders = _build_cat_feature_encoders(CATEGORICAL_FEATURES, GCS_BUCKET, MODEL_NAME)

In [ ]:
%%bigquery test_df --project $PROJECT_ID 

SELECT* EXCEPT (user_pseudo_id, churned, data_split)
FROM bqmlga4.ml_features
WHERE data_split="TEST"
LIMIT 3;

In [ ]:
# Converta registros de dataframe em dicionários de recursos para pré-processamento por nome de recurso.
test_instances = test_df.astype(str).to_dict(orient='records')

In [ ]:
# Aplique o pré-processamento para transformar recursos categóricos e retornar instâncias numéricas para previsão.
transformed_test_instances = preprocess_xgboost(test_instances, CATEGORICAL_FEATURES, feature_encoders)

In [ ]:
# Gere previsões do modelo implantado no Vertex AI Endpoint.
predictions = endpoint.predict(instances=transformed_test_instances)

In [ ]:
for idx, prediction in enumerate(predictions.predictions):
    # Rótulos de classe [1,0] recuperados de model_metadata.json no diretório do modelo GCS.
    # O padrão de classificação binária BQML é 0,5 com "Churn" acima e abaixo de "Not Churn".
    is_churned = "Churn" if prediction[0] >= 0.5 else "Not Churn"
    print(f"Prediction: Customer {idx} - {is_churned} {prediction}")
    print(test_df.iloc[idx].astype(str).to_json() + "\n")

## Próximos passos

Parabéns! Neste laboratório, você treinou, ajustou, explicou e implantou um modelo de desligamento de usuários do BigQuery ML para gerar previsões de desligamento on-line e em lote de alto impacto comercial para clientes-alvo com probabilidade de desligamento com intervenções como recompensas no jogo e notificações de lembrete.

Neste laboratório, você usou `user_psuedo_id` como um identificador de usuário. Como próximas etapas, você pode estender esse código ainda mais fazendo com que seu aplicativo retorne um `user_id` para o Google Analytics para que você possa juntar as previsões do seu modelo com dados primários adicionais, como histórico de compras e dados de engajamento de marketing. Isso permite integrar previsões em lote aos painéis do Looker para ajudar as equipes de produto a priorizar melhorias na experiência do usuário e as equipes de marketing criar intervenções direcionadas ao usuário, como e-mails de lembrete para melhorar a retenção.

Ao ter seu modelo no Vertex AI Prediction, você também tem um serviço de previsão escalável para chamar de seu aplicativo para integrar diretamente as previsões on-line, a fim de personalizar as experiências de jogo do usuário e permitir notificações direcionadas de criação de hábitos.

À medida que você coleta mais dados de seus usuários, convém avaliar regularmente seu modelo com dados novos e treinar novamente o modelo se perceber que a qualidade do modelo está diminuindo. O [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) pode ajudar você a automatizar, monitorar e controlar suas soluções de ML orquestrando seu fluxo de trabalho BQML sem servidor e armazenando os artefatos do seu fluxo de trabalho usando [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction). Para outra alternativa para modelos BQML contínuos, confira a postagem do blog [Continuous model assessment with BigQuery ML, Stored Procedures, and Cloud Scheduler](https://cloud.google.com/blog/topics/developers-practitioners/continuous-model-assessment-bigquery-ml-stored-procedures-and-cloud-scheduler).



## License

In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.